In [2]:
import numpy as np
import pandas as pd
import matplotlib.backends.backend_pdf
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

import scanpy as sc
import anndata as ad
import tangram as tg
import matplotlib.pyplot as plt
import seaborn as sns
import copy

In [3]:
project_location = "./"


In [4]:
# Load data

In [5]:
df_st = pd.read_csv(project_location + "data/merfish/merfishSpatial.csv", sep=" ")
coords = df_st['coord']
df_st.drop(columns=['coord'], inplace=True)
cellTypeCount =16
typeCountDf=  df_st.iloc[:,-cellTypeCount:]
# df_st = df_st.iloc[: , :-cellTypeCount]
df_st.head()

,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,ODImmature1,ODImmature2,Microglia,ODMature2,ODMature1,Endothelial3,ODMature3,ODMature4,Endothelial2,Ependymal
0,0.000000,0.000000,7.297361,0.000000,0.000000,8.209372,0.000000,0.912141,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
1,0.000000,0.000000,12.356107,14.387232,1.947237,48.028312,8.152290,17.650087,1.559171,0.388066,...,1,0,0,5,0,0,0,0,0,0
2,0.000000,0.000000,1.266888,2.486004,1.219116,4.845359,2.533784,1.573353,0.427118,2.650253,...,1,0,0,6,1,0,0,0,0,0
3,6.311896,9.177481,24.853501,20.157140,2.040983,16.319888,6.722705,4.156006,0.000000,1.400860,...,0,0,1,2,1,1,0,0,0,0
4,6.548440,4.512264,28.471008,73.614627,9.291271,34.175079,24.652650,25.305026,0.532110,0.665022,...,0,0,0,0,0,1,0,0,0,0


In [6]:
reference = pd.read_csv(project_location + "data/merfish/markerGene_for_merfish_data.csv")

reference.drop(columns=['p_value'], inplace=True)
# reference.drop(range(148,168))
reference.drop(reference.index[reference.cell_type=='EpendymalInhibitory'].tolist(), inplace=True)
reference['cell_type'] = reference['cell_type'].astype(str).str.replace(" ", "")
reference.head()

,cell_type,marker_gene
0,Astrocyte,Aldh1l1
1,Astrocyte,Aqp4
2,Astrocyte,Cxcl14
3,Astrocyte,Mlc1
4,Astrocyte,Ttyh2


In [7]:
genes = df_st.columns[:155]
markers = reference.groupby('cell_type').agg(list).marker_gene
celltypes = reference.cell_type.unique().tolist()
expressions = df_st.drop(columns=celltypes)
expressions = MinMaxScaler().fit_transform(expressions)
counts_st = pd.DataFrame(expressions, columns=genes)
cellcount = pd.DataFrame(np.sum(df_st[celltypes], axis=1), columns=['cellcount'])

## Single-cell data

In [8]:
df_sc = pd.read_csv(project_location + "data/merfish/merfishVisium.csv")
df_sc.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
0,1f9a8c19-b089-43d1-b609-7e791dc2c70f,1,Female,Naive,-0.24,-3749.176078,-3749.458442,Astrocyte,NaN,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.273327,0.0,0.003225,0.000000
1,b13e98f4-5c2b-4e96-985e-3e93aedc7221,1,Female,Naive,-0.24,-3746.226210,-3742.259347,Inhibitory,I-1,0.0,...,0.097542,0.0,0.000000,0.0,0.000000,0.016809,0.293094,0.0,0.006602,0.018258
2,d06cb29e-10ee-4bbc-b74e-90237999ef4b,1,Female,Naive,-0.24,-3742.897643,-3790.648737,Inhibitory,I-13,0.0,...,0.087302,0.0,0.063460,0.0,0.072791,0.000000,0.289555,0.0,0.000000,0.018325
3,12e2a165-57c7-4f37-96dd-23f6574af4ba,1,Female,Naive,-0.24,-3738.423005,-3773.259265,Inhibitory,I-19,0.0,...,0.098912,0.0,0.000000,0.0,0.048410,0.000000,0.247843,0.0,0.000000,0.008280
4,14a0f396-b13d-4d45-89a3-86c2047bf3f9,1,Female,Naive,-0.24,-3736.407620,-3895.590306,Pericytes,NaN,0.0,...,0.047054,0.0,0.178222,0.0,0.000000,0.000000,0.294404,0.0,0.000000,0.000000


In [9]:
counts_sc = df_sc[genes]

annotation= df_sc["Cell_class"]
frequency={}
for item in annotation:
    if item in frequency:
        frequency[item]+=1
    else:
        frequency[item]=1

frequency


{'Astrocyte': 745,
 'Inhibitory': 2219,
 'Pericytes': 62,
 'Ambiguous': 869,
 'Endothelial1': 301,
 'Excitatory': 1219,
 'ODImmature1': 223,
 'ODImmature2': 7,
 'Microglia': 106,
 'ODMature2': 241,
 'ODMature1': 49,
 'Endothelial3': 117,
 'ODMature3': 6,
 'ODMature4': 10,
 'Endothelial2': 19,
 'Ependymal': 219}

In [10]:
excitatoryDf= df_sc.loc[df_sc['Cell_class']=="Excitatory"]
excitatoryDf =excitatoryDf[genes]
InhibitoryDf=df_sc.loc[df_sc['Cell_class']=="Inhibitory"]
InhibitoryDf=InhibitoryDf[genes]

options=["Excitatory","Inhibitory"]
rsltSingleCell= df_sc[df_sc["Cell_class"].isin(options)]


rsltSingleCell.to_csv("./data/FaultSingleCell.csv",index= False)

In [11]:
rsltSingleCell.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
1,b13e98f4-5c2b-4e96-985e-3e93aedc7221,1,Female,Naive,-0.24,-3746.226210,-3742.259347,Inhibitory,I-1,0.0,...,0.097542,0.0,0.00000,0.0,0.000000,0.016809,0.293094,0.000000,0.006602,0.018258
2,d06cb29e-10ee-4bbc-b74e-90237999ef4b,1,Female,Naive,-0.24,-3742.897643,-3790.648737,Inhibitory,I-13,0.0,...,0.087302,0.0,0.06346,0.0,0.072791,0.000000,0.289555,0.000000,0.000000,0.018325
3,12e2a165-57c7-4f37-96dd-23f6574af4ba,1,Female,Naive,-0.24,-3738.423005,-3773.259265,Inhibitory,I-19,0.0,...,0.098912,0.0,0.00000,0.0,0.048410,0.000000,0.247843,0.000000,0.000000,0.008280
8,cfe60a3a-92a7-46d6-a2cf-687fd2af96fc,1,Female,Naive,-0.24,-3717.144006,-3760.757175,Excitatory,E-2,0.0,...,0.078634,0.0,0.00000,0.0,0.050427,0.000000,0.295925,0.000000,0.000000,0.000000
21,3c220612-a7aa-4cab-a5bc-f5f2cb910687,1,Female,Naive,-0.24,-3681.886968,-3796.560642,Inhibitory,I-1,0.0,...,0.007378,0.0,0.00000,0.0,0.000000,0.000000,0.268177,0.014588,0.016645,0.000000


In [12]:
spotCount=200
randCount= np.random.randint(1,6,spotCount)
randCount

array([5, 5, 5, 2, 5, 5, 1, 3, 5, 3, 3, 5, 2, 2, 4, 5, 3, 1, 4, 2, 5, 5,
       1, 5, 5, 1, 5, 3, 1, 4, 5, 2, 3, 3, 3, 3, 4, 4, 2, 5, 1, 4, 2, 2,
       5, 3, 4, 2, 5, 2, 3, 3, 4, 3, 3, 2, 5, 2, 1, 4, 1, 1, 5, 1, 5, 2,
       3, 4, 5, 5, 4, 3, 5, 3, 1, 5, 3, 1, 1, 1, 1, 5, 3, 4, 2, 2, 3, 4,
       5, 1, 5, 4, 1, 4, 5, 1, 4, 5, 3, 5, 1, 1, 1, 2, 2, 5, 4, 2, 5, 3,
       4, 3, 2, 4, 2, 4, 4, 5, 3, 1, 4, 3, 3, 3, 4, 4, 5, 2, 1, 5, 1, 1,
       1, 3, 3, 4, 4, 5, 2, 2, 1, 3, 1, 2, 5, 4, 5, 1, 1, 4, 5, 5, 2, 5,
       5, 3, 1, 4, 3, 1, 5, 4, 2, 4, 2, 2, 4, 3, 4, 2, 4, 5, 1, 1, 4, 4,
       2, 4, 4, 4, 4, 1, 1, 3, 4, 3, 1, 3, 4, 4, 5, 3, 4, 2, 1, 5, 5, 5,
       1, 2])

In [27]:

count=0
spatialData= pd.DataFrame(columns=genes)

# exRows=excitatoryDf[count:(count+randCount[0])]
# inRows= InhibitoryDf[count:(count+randCount[0])]
# exRows = pd.concat([exRows,inRows])
# spot= exRows.sum(axis=0)    
# print(spot)


for i in range(spotCount):
    exRows=excitatoryDf[count:(count+randCount[i])]
    inRows= InhibitoryDf[count:(count+randCount[i])]
    exRows = pd.concat([exRows,inRows])
    
    spot= exRows.sum(axis=0)    
    spatialData = spatialData.append(spot,ignore_index=True)
    
    count+=randCount[i]
spatialData["Excitatory"]= randCount
spatialData["Inhibitory"] =randCount
spatialData.head()



stringVal=[]
for i in range(spotCount):
    res= str(i)+"x"+ str(i)
    stringVal.append(res)
    # print(spatialData.iloc[i]["coord"])
    # # print(res)
    # spatialData.iloc[i]["coord"]= " hello"
spatialData.insert(0, 'coord', stringVal)
spatialData["coord"]=spatialData.coord.astype(str)

spatialData.to_csv("./data/FaultSpatial.csv",index= False)
# print(spatialData.iloc[0]["coord"])

In [28]:
spatialData.head()

,coord,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,...,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf,Excitatory,Inhibitory
0,0x0,0.360787,0.516422,17.601368,76.883393,2.953295,27.230260,22.420963,45.710406,2.156298,...,0.757329,0.0,0.378962,0.166447,2.591885,0.762383,0.046929,0.094774,5,5
1,1x1,1.737130,2.260192,7.648333,106.473509,3.803570,7.554986,28.754944,51.391699,0.886948,...,0.934362,0.0,0.257981,0.213244,3.299016,1.658193,0.000000,0.485146,5,5
2,2x2,1.191190,1.257806,15.552129,131.368792,4.367850,22.890013,28.395882,53.620005,0.000000,...,0.708458,0.0,0.292812,1.323163,2.638216,0.862516,0.002071,0.204437,5,5
3,3x3,0.569134,0.000000,3.893906,55.749210,2.723574,1.930917,9.559152,26.016232,0.892213,...,0.000000,0.0,0.000000,0.023781,1.003819,1.006273,0.000000,0.269401,2,2
4,4x4,0.410587,1.475133,26.930355,63.826621,3.529038,17.849860,32.596714,86.288442,2.328086,...,3.202039,0.0,0.728464,2.040549,2.396148,0.000000,0.000000,0.793432,5,5
